---
# **Tarefa 8 - Projeto Computacional**


**MS428A - 30/09/2021**

Alunos:

Elizeu Caetano Da Silva RA:233817

Enrico Campos de Mira RA: 196437

---

## **Introdução**

---



Definimos um problema de programação linear como sendo :

Dado um vetor x ∈ R<sup>n</sup>, um vetor de variável, um conjunto A denominado conjunto de factibilidade e f(x) função objetivo , desejamos encontrar um valor x que minimiza f em A, e se tal valor existe dizemos que é a solução ótima para o problema. Se existe uma solução ótima então existe um vértice ótimo.

Dado um problema de programação Linear , o algoritmo Simplex busca vértices ótimos através de comandos iterativos. Na prática o método inicia com uma base factível inicial (que corresponde a um ponto da fronteira de A) e verifica se este é ótimo através de cálculos numéricos e a partir dele salta para outro extremo de menor valor ; assim sucessivamente até encontrar o X que minimiza f.

No caso em que não há uma base factível trivial (Identidade de ordem m) lança-se mão de dois métodos para iniciar o simplex , o método do Big M ou das duas fases.

O PL pode ser escrito em sua forma matricial como

min f(x):
s.a {Ax = B e  $x \ge 0 $}

Utilizamos a linguagem Python para programação e importamos a biblioteca Numpy para facilitar a implementação; ao longo do código é explicado cada passo que define o método Simplex

### **Entradas**:

*   m: número de restrições (linhas);
*   n: número de variáveis* (colunas);
*   c: vetor dos custos;
*   b: vetor dos recursos (ou termos independentes);
*   A: matriz dos coeficientes das restrições (com as variáveis de folga).

A ∈ R<sup>m×n</sup>,
b ∈ R<sup>m</sup>,
c ∈ R<sup>n</sup>,
m ∈ R e n ∈ R.

*Incluindo as variáveis de folga

### **Saídas**:

Se o problema for ilimitado:
*   exibir essa informação


Se o problema tiver solução, exibir:


*   x: vetor das variáveis de decisão, que é a solução  ótima do PPL;
*   F(x): valor  ótimo da solução.

x ∈ R<sup>n</sup> e F(x) ∈ R.



## **Código Explicado**

---

### **Simplex**

---

A função SIMPLEX definida abaixo foi definida com base no método das duas fases. Ao ser executada, a função utiliza a função FASEI para encontrar um vetor com os índices das variáveis básicas. Caso o encontre uma solução básica factível, o código seguirá para a função FASEII para resolver o problema. A função FASEII resolverá o problema da fase 2

Caso não encontre uma solução básica factível na FASEI a função para e retorna a resposta

A função Simplex recebe como entrada os argumentos (A,b,c, m,n) que correspondem as entradas enunciadas acima na introdução(no formato array, como nos exemplos abaixo).

In [ ]:
# Importando Pacotes:
import numpy as np

# Simplex
def Simplex(A,b,c, m,n):
  # Verifica se todos os valores de b são positivos
  (A,b) = VERIFICA_b(A,b)
  # Simplex
  v = FASEI(A,b,m,n)
  if  v.any() != 0:
    FASEII(c,b,A, v)

**Verificando b**

A função VERIFICA_b(A,b) recebe a Matriz A e o vetor b e, a partir disso, verifica se b possue algum elemento menor que 0. Caso ocorra, o algoritmo seleciona o elemento em b e sua respectiva linha em A e multiplica ambos por -1

In [ ]:
def VERIFICA_b(A,b):

  if any(b < 0):
    A[b<= 0,:] = A[b< 0,:] *(-1)
    b[b<= 0] = b[b< 0] *(-1)

  return A,b

### **FASE  I**

---

Para o a implementação da fase 1 utilizamos o método das duas fases. Para isso primeiramente construímos o problema auxíliar e, em seguida, utilizamos o algoritmo da FASE II para resolver o problema.

Após a resolução do problema auxíliar, o programa nos retornará um vetor com os índices das variáveis que compõem a base factível encontrada (v). Ou caso não encontre uma base factível, um vetor nulo(para para a função SIMPLEX) e a mensagem: "O problema não possui solução factível"





In [ ]:
def FASEI(A,b,m,n):

  # Vetor de custos do problema auxíliar
  c = np.append(np.zeros(n, int),np.ones(m, int))
  # Matriz A do problema auxíliar
  A = np.append(A,np.identity(m,int), axis = 1)
  # Base inicial do problema auxíliar
  v = np.array(range(n ,n+m))

  while(True):

      # FASE II: { início da iteração simplex }
      (B, cb, N, cn) = PASSO_0(A, b, c, v)

      # Passo 1: { cálculo da solução básica }
      (xb, Fxb) = PASSO_1(B, cb, b)

      # Chegando factibilidade (xb >= 0)
      if NAO_FACTIVEL(xb):
        print("O problema não possui solução factível")
        v = np.array(0)
        return v
        break

      # Passo 2: { cálculo dos custos relativos }
      (custos_relativos, var_entra) = PASSO_2(B, N, cb, cn)

      # Passo 3: { teste de otimalidade }
      if PASSO_3(custos_relativos):
        return v
        break

      # Passo 4: { cálculo da direção simplex }
      y =PASSO_4(B,N,var_entra)

      if NAO_FINITA(y):
        print("O problema não possui solução ótima finita")
        break

      # Passo 5: { determinar passo e variável a sair da base }
      var_sai = PASSO_5(xb, y)

      # Passo 6: { atualização: nova partição básica }
      v = PASSO_6(c, v, var_entra, var_sai)


### **FASE II**
---

A função FASEII resolverá o problema da fase 2. Para isso, a função receberá o vetor de custos (c), os valores de b (b), a matriz de coeficientes (A) e o vetor com o índice das varíaveis básicas encontrada na FASEI(v).

Posto isso, o programa seguirá resolvendo o problema da Fase II. Para isso, chamará as funções abaixo para realizar cada passo do algoritmo.

A função retornará se o problema é ilimitado ou possui uma solução única. Nesse último caso retornará o vetor xb e o valor de F(xb)

In [ ]:
def FASEII(c, b, A, v):
  while(True):

    # FASE II: { início da iteração simplex }
    (B, cb, N, cn) = PASSO_0(A, b, c, v)

    # Passo 1: { cálculo da solução básica }
    (xb, Fxb) = PASSO_1(B, cb, b)

    # Passo 2: { cálculo dos custos relativos }
    (custos_relativos, var_entra) = PASSO_2(B, N, cb, cn)

    # Passo 3: { teste de otimalidade }
    if PASSO_3(custos_relativos):
      x = np.zeros(len(cb)+len(cn))
      x[v]= xb
      print("Solução ótima:","\n x =", end = ' ')
      for i in x:
        print(i, end = '   ')
      print("\n F(x)=" , Fxb)
      break

    # Passo 4: { cálculo da direção simplex }
    y =PASSO_4(B,N,var_entra)

    if NAO_FINITA(y):
      print("O problema não possui solução ótima finita")
      break

    # Passo 5: { determinar passo e variável a sair da base }
    var_sai = PASSO_5(xb, y)

    # Passo 6: { atualização: nova partição básica }
    v = PASSO_6(c, v, var_entra, var_sai)



**Passo 0: { início da iteração simplex }**

No passo 0, recebemos o vetor de custos (c), os valores de b (b), a matriz de coeficientes (A) e o vetor com o índice das varíaveis básicas encontrada na FASEI (v). A partir dessas informações a função PASSO_0 nos retorna a matriz das variáveis básicas(B), o vetor de custos básicos(cb), matriz das variáveis não básicas(N) e o vetor de custo das variáveis não básicas (cn).

In [ ]:

# FASE II: { início da iteração simplex }
def PASSO_0(A, b, c, v):
  B = A[:,v]
  cb = c[v]
  u = np.setxor1d(np.array(range(len(c))),v)
  cn = c[u]
  N = A[:,u]
  return B, cb, N, cn



**Passo 1: { cálculo da solução básica }**

$x$ <sub>b</sub> = $B$<sup>-1</sup> $* b$ (resolvendo o sistema $ Bx$ <sub>b</sub> $= b$)

$x$ <sub>n</sub>  = 0


Calculo do valor da função atual: $F$($x$ <sub>b</sub> ) = $c$ <sub>b</sub>  * $x$ <sub>b</sub>  

In [ ]:
# Passo 1: { cálculo da solução básica }
def PASSO_1(B, cb, b):

  xb = np.linalg.solve(B, b)
  Fxb = np.dot(cb,xb)

  return xb, Fxb

**Verificando Factibilidade**

A função NAO_FACTIVEL verifica se a solução encontrada é factível. Essa função será utilizada já na Fase I.

Caso não seja factível nos retorna True, caso contrário False


In [ ]:
def NAO_FACTIVEL(xb):
  if any(xb < 0):
      return True
  else:
      return False

**Passo 2: { cálculo dos custos relativos }**

*2.1) { cálculo do vetor multiplicador simplex }*

λ<sup>T</sup> = c<sub>B</sub><sup>T</sup>  * B<sup>-1</sup>  (resolvendo o sistema B<sup>T</sup> λ = c<sub>B )

*2.2) { custos relativos }*

$\hat{c}$<sub>Nj</sub> = $c$<sub>Nj</sub> − λ<sup>T</sup>  * $\hat{a}$<sub>Nj</sub>  , j = 1, 2, . . . , n − m.

*2.3) { escolha da variável a entrar na base }*

$\hat{c}$<sub>Nk</sub> = min {$\hat{c}$<sub>Nj</sub> , j = 1, . . . , n − m}. A variável $x$<sub>Nk</sub> entra na base.

**Retorna:**

$\hat{c}$<sub>Nj</sub>(o vetor de custos_relativos) e $x$<sub>Ak</sub> (o índice da varíavel que entrará na base)

In [ ]:
# Passo 2: { cálculo dos custos relativos }

def PASSO_2(B, N, cb, cn):
  # 2.1) { cálculo do vetor multiplicador simplex }
  Lambda = np.linalg.solve(np.transpose(B) , cb)

  # 2.2) { custos relativos }
  custos_relativos = []
  for i in range(len(cn)):
      custos_relativos.append(cn[i] - np.dot(np.transpose(Lambda) , N[:,i]))

  # 2.3) { escolha da variável a entrar na base }
  var_entra = custos_relativos.index(min(custos_relativos))

  return custos_relativos, var_entra

**Passo 3: { teste de otimalidade }**

Se $\hat{c}$<sub>Nk</sub>  ≥ 0, então PARE = VERDADE { solução atual é ótima }

In [ ]:
# Passo 3: { teste de otimalidade }
def PASSO_3(custos_relativos):
    if all(x >= 0 for x in custos_relativos):
      return True
    else:
      return False

**Passo 4: { cálculo da direção simplex }**

$y=B$<sup>-1</sup>  * $a$<sub>Nk</sub> (resolva o sistema $B y$ = $a$<sub>Nk</sub> )

In [ ]:
# Passo 4: { cálculo da direção simplex }
def PASSO_4(B,N,var_entra):
  y = np.linalg.solve(B , N[:,var_entra])
  return y

**Passo 5: {verificar se o problema tem solução finita}**

Se $y ≤ 0$, então PARE = VERDADE { problema não tem solução  ótima finita }

A função NAO_FINITA nos retorna True se o problema não possui solução finita ou False se o problema possui solução finita


In [ ]:
def NAO_FINITA(y):
  if all(y <= 0):
      return True
  else:
      return False

**Passo 5: { determinar passo e variável a sair da base }**

$\hat{ε}$<sub>Nk</sub> = $\frac{\hat{x}_{B_{l}}}{{y}_{l}}$ = min {$\frac{\hat{x}_{B_{i}}}{{y}_{i}}$, i = 1, . . . , m}. A variáel $x_{B_{l}}$ sai da base.


In [ ]:
# Passo 5: { determinar passo e variável a sair da base }
def PASSO_5(xb, y):
  epsilon =   min(xb[y > 0 ] / y[y > 0 ])

  var_sai =  np.where(np.divide(xb, y, out=np.zeros_like(y), where=y!=0) == epsilon )

  return var_sai

**Passo 6: { atualização: nova partição básica }**

 $a$<sub>Blk</sub> $←→$  $a$<sub>Nk</sub>

A função fara isso atualizando o vetor v

In [ ]:
# Passo 6: { atualização: nova partição básica }
def PASSO_6(c, v, var_entra, var_sai):
  u = np.setxor1d(np.array(range(len(c))),v)
  v = np.union1d(np.setxor1d(v,v[var_sai]),u[var_entra])
  return v


## **Exemplos**

---


**Tarefa 6 - Método primal Simplex (Fase II)**

MS428A - 21/09/2021

Data de entrega: 28/09/2021

---
Considere o PL a seguir:

min f(x) = −x1 − 3x2

sujeito a:

−3x1 + 4x2 ≤ 12

x1 − x2 ≤ 4

x1 + x2 ≤ 6

x1 ≥ 0, x2 ≥ 0

(a) Resolva pelo algoritmo Simplex (dado na Aula 10). Não omita passos, exiba todas as contas.

---

In [ ]:
#Tarefa 6 - Método (primal) simplex - Fase II

# Dados enunciados
c = np.array([-1, -3, 0, 0, 0])
A = np.array([
    [-3, 4, 1, 0, 0],
    [ 1, -1, 0, 1, 0],
    [ 1, 1, 0, 0, 1]
])
b =np.array( [12, 4, 6])
(m,n) = (3,5)

# Simplex
Simplex( A,b,c,m,n)

Solução ótima: 
 x = 1.7142857142857153   4.2857142857142865   0.0   6.571428571428571   0.0   
 F(x)= -14.571428571428575


---
**Atividade C - Método Primal Simplex**

MS428A - 07/10/2021

---

Considere o seguinte PL:

Min f(x) = −2x1 − x2

sujeito a:

−2x1 + x2 ≤ 2

−2x1 + 3x2 ≥ −6

x1 ≥ 0, x2 ≥ 0

(a) Resolva o PL utilizando o algoritmo Primal Simplex (visto na Aula 10). Não omita passos
e exiba todas as contas.

---

In [ ]:
# Atividade C

# Dados enunciados
c = np.array([-2, -1, 0, 0])
A = np.array([
    [-2, 1, 1, 0],
    [ -2, 3, 0, -1]
])
b =np.array( [2, -6])
(m,n) = (2,4)

# Simplex
Simplex( A,b,c,m,n)

O problema não possui solução ótima finita


---

**Atividade D - Método Primal Simplex**

MS428A - 14/10/2021

---

Resolva o PL a seguir utilizando o algoritmo Primal Simplex. Não omita passos e exiba todas as contas. Deixe claro qual método você escolheu para a Fase I.

Min f(x) = −2x1 + 0,5x2

sujeito a:

x1 + 3x2 ≥ 3

2x1 + 3x2 ≤ 12

x1 ≥ 0, x2 ≥ 0

---

In [ ]:
# Atividade D

# Dados enunciados
c = np.array([-2, 0.5, 0, 0])
A = np.array([
    [1, 3, -1, 0],
    [ 2, 3, 0, 1]
])
b =np.array( [3, 12])
(m,n) = (2,4)

# Simplex
Simplex( A,b,c,m,n)

Solução ótima: 
 x = 6.0   0.0   3.0   0.0   
 F(x)= -12.0


---

**Tarefa 7 - Método primal Simplex (Fase I)**

MS428A - 28/09/2021
Data de entrega: 05/10/2021

---

Resolva os PLs a seguir. Escolha um deles para resolver pelo m ́etodo das duas fases (problema auxiliar)
e o outro, pelo m ́etodo Big-M. N ̃ao omita passos, exiba todas as contas.

(a)
max f(x) = 3x1 − 4x2

sujeito a:

x1 + x2 ≤ 4

2x1 + 3x2 ≥ 18

x1 ≥ 0, x2 ≥ 0

(b)
min f(x) = 2x1 + x2

sujeito a:

x1 + 6.5x2 ≥ 5

2x1 + x2 ≥ 4

5x1 + 4x2 ≤ 20

x1 ≥ 0, x2 ≥ 0

Dica: Lembre-se de que para aplicar o algoritmo o problema deve estar na forma padrão.

---

In [ ]:
#Tarefa 7 - Método primal Simplex (Fase I)
#(A)

# Dados enunciados
c = np.array([3, -4, 0, 0])
A = np.array([
    [1, 1, 1, 0],
    [ 2, 3, 0, 1]
])
b =np.array( [4,18])
(m,n) = (2,4)

# Simplex
Simplex( A,b,c,m,n)

Solução ótima: 
 x = 0.0   4.0   0.0   5.999999999999999   
 F(x)= -16.0


In [ ]:
#(B)

# Dados enunciados
c = np.array([1, 2, 0, 0, 0])
A = np.array([
    [1, 6.5, -1, 0, 0],
    [ 2, 1, 0, -1, 0],
    [ 5, 4, 0, 0, 1]
])
b =np.array( [5,4,20])
(m,n) = (3,5)

# Simplex
Simplex( A,b,c,m,n)

Solução ótima: 
 x = 1.75   0.5   0.0   0.0   9.25   
 F(x)= 2.75


In [ ]:
# Dados enunciados
c = np.array([-2.5,-1,0,0])
A = np.array([
            [3,5, 1, 0],
            [5, 2, 0, 1]
])
b =np.array( [15,10])
(m,n) = (2,4)

# Simplex
Simplex( A,b,c,m,n)


Solução ótima: 
 x = 1.0526315789473684   2.368421052631579   0.0   0.0   
 F(x)= -5.0


In [ ]:
# Exemplo sem solução finita

# Dados enunciados
c = np.array([-3,-3,0,0])
A = np.array([
            [1,-1, -1, 0],
            [0.5, -1, 0, -1]
])
b =np.array( [1,2])
(m,n) = (2,4)
v=np.array( [-2,-3])

# Simplex
Simplex(A, b,c, m, n)


O problema não possui solução ótima finita


In [ ]:
# Problema em maior escala

# Dados enunciados
c = np.array([40,32,35,36,37,0,0,0,0,0,0])
A = np.array([
            [2, 3, 1.5, 10, 2.25, 1, 0, 0, 0, 0, 0],
            [4, 3, 5, 1, 4, 0, 1, 0, 0, 0, 0],
            [3, 2.5, 2.5, 2, 2.5, 0, 0, 1, 0, 0, 0],
            [2, 1, 2, 1, 2, 0, 0, 0, 1, 0, 0],
            [2, 2, 3, 3, 4, 0, 0, 0, 0, 1, 0],
            [2, 3, 2, 2, 3, 0, 0, 0, 0, 0, 1]
])
b =np.array( [650, 700, 700, 300, 450, 550])
(m,n) = (6,11)

Simplex(A,b,c,m,n)

Solução ótima: 
 x = 0.0   0.0   0.0   0.0   0.0   650.0   700.0   700.0   300.0   450.0   550.0   
 F(x)= 0.0
